In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext() 
ssc = StreamingContext(sc, 5)
lines = ssc.socketTextStream("localhost", 9000)

C:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


----

In [3]:
import numpy as np
from scipy import sparse

def get_vals(w: int = 5):
    coefs = np.random.randint(1, 1000, (w,w))
    p = np.random.randint(3000, 100000, w)
    return coefs, p + ~(p % 2).astype(bool)

def fit_poly(row, x):
    return np.poly1d(row)(x)

def cm_multihash(x: int, counts: int, coefs, p, const: int = 2719):
    table = np.zeros((p.shape[0], const))
    j = ((np.apply_along_axis(fit_poly, 1, coefs, x) % p) % const).astype(int)
    table[np.arange(0, p.shape[0]),j] = counts
    return sparse.csr_matrix(table)

def sparse_sim(sk1: sparse.csr.csr_matrix, sk2: sparse.csr.csr_matrix):
    return sk1.multiply(sk2).sum(axis = 1).min()

In [4]:
coefs, p = get_vals()

----

In [9]:
tau = 3000

In [5]:
collect_hashes = lines.window(60).map(lambda line: ((line.split(',')[0], line.split(',')[1]), 1))\
                        .reduceByKey(lambda x,y: x+y)\
                        .map(lambda line: (line[0][0], cm_multihash(int(line[0][1]), line[1], coefs, p)))\
                        .reduceByKey(lambda x,y: x+y)

In [6]:
prepared = collect_hashes.map(lambda line: (1, (line[0], line[1])))
combo = prepared.transform(lambda RDD: RDD.join(RDD)\
                                          .filter(lambda line: int(line[1][0][0]) > int(line[1][1][0]))\
                                          .map(lambda line: [line[0], sparse_sim(line[1][0][1], line[1][1][1])])\
                                          .filter(lambda line: line[1] >= tau)
                           
#prepared.pprint(1)
combo.pprint(10)

WORKING NOW: 10 rows per second 

In [7]:
ssc.start()

-------------------------------------------
Time: 2022-03-24 19:20:10
-------------------------------------------

-------------------------------------------
Time: 2022-03-24 19:20:15
-------------------------------------------
(1, (('44', <5x2719 sparse matrix of type '<class 'numpy.float64'>'
	with 25 stored elements in Compressed Sparse Row format>), ('44', <5x2719 sparse matrix of type '<class 'numpy.float64'>'
	with 25 stored elements in Compressed Sparse Row format>)))
...

-------------------------------------------
Time: 2022-03-24 19:20:20
-------------------------------------------
(1, (('44', <5x2719 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>), ('44', <5x2719 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>)))
...

-------------------------------------------
Time: 2022-03-24 19:20:25
-------------------------------------------
(1, (('44', <5x2719 sparse ma

In [8]:
ssc.stop()